# [Module 4.3.2] inference on custem tf serving image


---
이 노트북은 약 10정도 시간이 소요 됩니다.

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
%store -r 

In [3]:
# training_job_name = 'bert2tweet-2020-07-08-07-58-27-895'
print(training_job_name)

tensorflow-training-2020-07-26-06-47-18-612


In [4]:
# inference_image = '057716757052.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu'
inference_image = ecr_infer_custom_image_tf_serving_20_cpu
print("inference_image: ", inference_image)

inference_image:  057716757052.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu


## Model Creation with Custome TFS Docker Image and Inference code

In [5]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              entry_point='inference.py',
              image = inference_image
             ) 

## 엔드포인트 생성

In [6]:
%%time

instance_type='ml.m4.xlarge'
deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


-------------!CPU times: user 1min 2s, sys: 10.4 s, total: 1min 12s
Wall time: 7min 42s


## Predictor Creation on the Endpoint

In [7]:
# tweet_bert_endpoint_name = 'train_text, train_label, test_text, test_label = tweet_data.split_train_test_data(texts, labels, 0.9)
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

sagemaker-tensorflow-serving-2020-07-26-08-23-00-115


In [8]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                     )

## Inference 실행

In [9]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
None


In [10]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
1974,ter stegen i have gladbach's crest in my hear...,0
1530,you playing too much,1
3515,we should be atleast 11 0 up by now,1
639,hoe my god,1
3955,the sass is so real,1
4471,look at mitrovic,1
4236,no money in the world would make me clean the ...,1
2201,loving the cape town weather sasfinest,0
3745,it's so funny how you drink now,1
949,i'm ded,1


In [11]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list


## Top 3 이모티콘 추천

In [13]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 3
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]



    predicted_classes = predictor.predict(reviews)[0]
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    
    print('Prediction: {},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2])                                       
        ))    
        



tweet:  ter stegen i have gladbach's crest in my heart but now there are two clubs in my heart and i'm happy about that h 
Ground_truth- 0:❤
 
Prediction: 5,None,6,None,3,None 
 
tweet:  you playing too much  
Ground_truth- 1:😂
 
Prediction: 3,None,6,None,5,None 
 
tweet: we should be atleast 11 0 up by now 
Ground_truth- 1:😂
 
Prediction: 6,None,3,None,5,None 
 
tweet: hoe my god  
Ground_truth- 1:😂
 
Prediction: 3,None,6,None,5,None 
 
tweet:  the sass is so real 
Ground_truth- 1:😂
 
Prediction: 6,None,3,None,5,None 
 
tweet:  look at mitrovic 
Ground_truth- 1:😂
 
Prediction: 3,None,6,None,5,None 
 
tweet: no money in the world would make me clean the toilets at festivals never seen anything like it than i did at ep animals 
Ground_truth- 1:😂
 
Prediction: 5,None,6,None,3,None 
 
tweet: loving the cape town weather sasfinest 
Ground_truth- 0:❤
 
Prediction: 6,None,3,None,5,None 
 
tweet:  it's so funny how you drink now  
Ground_truth- 1:😂
 
Prediction: 6,None,3,None,5,None 
 
tweet: